## Import libraries
Import libraries to perform one hot coding and rescaling of features

In [69]:
## Import librariries
import sys, os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump, load
import pickle
from typing import List, Tuple
from datetime import datetime
from dateutil import parser

## Import raw features
Import the raw feature file and inspect it.

In [70]:
# Read the raw feature CSV file into a pandas DataFrame
## for raw_features, use this path: /Users/adityaponnada/Downloads/time_study_data/raw_features_rnn.csv
## for heldout data features, use this path: /Users/adityaponnada/Downloads/time_study_data/heldout_raw_features.csv
raw_feature_df = pd.read_csv("/Users/adityaponnada/Downloads/time_study_data/raw_features_rnn.csv")
print(f"Raw feature DataFrame shape: {raw_feature_df.shape}")
raw_feature_df.head()

Raw feature DataFrame shape: (1088856, 23)


,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
0,afflictedrevenueepilepsy@timestudy_com,2021-06-23 06:38:01,0,0,Early Morning,0.0,0.0,Home,0,0.006074,...,981.983333,8.016667,5.545853,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,afflictedrevenueepilepsy@timestudy_com,2021-06-23 06:46:02,1,0,Early Morning,NaN,NaN,Home,0,0.005902,...,973.966667,16.033333,24.657388,0,0.000000,0.000000,8.016667,0.000000,0.000000,0.000000
2,afflictedrevenueepilepsy@timestudy_com,2021-06-23 06:54:04,1,0,Early Morning,0.0,0.0,Home,0,0.005426,...,965.933333,24.066667,0.000000,0,0.500000,0.500000,8.033333,8.033333,0.500000,0.500000
3,afflictedrevenueepilepsy@timestudy_com,2021-06-23 07:12:02,0,0,Early Morning,0.0,1.0,Home,0,0.005985,...,947.966667,42.033333,1.005139,0,0.666667,0.666667,17.966667,17.966667,0.666667,0.666667
4,afflictedrevenueepilepsy@timestudy_com,2021-06-23 07:23:02,0,0,Early Morning,0.0,1.0,Home,0,0.006400,...,936.966667,53.033333,0.044360,0,0.500000,0.500000,11.000000,28.966667,0.500000,0.500000


In [71]:
raw_feature_df['wake_day_part'].value_counts()

wake_day_part
1.0    289100
2.0    284747
0.0    257866
3.0    255129
Name: count, dtype: int64

In [72]:
raw_feature_df.columns

Index(['participant_id', 'prompt_time_converted', 'outcome', 'is_weekend',
       'time_of_day', 'in_battery_saver_mode', 'charging_status',
       'location_category', 'screen_on', 'dist_from_home', 'is_phone_locked',
       'last_phone_usage', 'wake_day_part', 'closeness_to_sleep_time',
       'closeness_to_wake_time', 'mims_5min', 'days_in_study',
       'completion_24h', 'completion_1h', 'time_between_prompts',
       'time_since_last_answered', 'completion_since_wake',
       'completion_since_start'],
      dtype='object')

In [73]:
# Print unique participant_id values from raw_feature_df (count + sample)
if 'raw_feature_df' in globals():
    vals = raw_feature_df['participant_id'].dropna().unique()
    print(f'Unique participant_id count: {len(vals)}')
    try:
        full_list = sorted(vals)
    except Exception:
        full_list = list(vals)
    for i, pid in enumerate(full_list, start=1):
        print(f'{i:3d}: {pid}')
else:
    print('raw_feature_df not found. Run the CSV load cell first.')


Unique participant_id count: 100
  1: afflictedrevenueepilepsy@timestudy_com
  2: anagramprobingscrooge@timestudy_com
  3: anthillfastinglucrative@timestudy_com
  4: arrivejanitoruniformly@timestudy_com
  5: attirecrabbinghumbling@timestudy_com
  6: backfirebankedprudishly@timestudy_com
  7: badlandwiltmuseum@timestudy_com
  8: bannisterhardwiredladle@timestudy_com
  9: bartenderradiatorapplied@timestudy_com
 10: brinkaminounframed@timestudy_com
 11: catsupexploitmocker@timestudy_com
 12: caucuscattlemockup@timestudy_com
 13: certifiedembargobartender@timestudy_com
 14: chewingslouchingfailing@timestudy_com
 15: congresscyclistdefender@timestudy_com
 16: copybrickcreative@timestudy_com
 17: coynessculminatebarista@timestudy_com
 18: craftworkattendeeensnare@timestudy_com
 19: debatableuneasyeveryone@timestudy_com
 20: defilinganywayimmovable@timestudy_com
 21: diagramuncoupleoutput@timestudy_com
 22: dissuadecelestialrelic@timestudy_com
 23: distresslitigatemassager@timestudy_com
 24: 

In [74]:
# Observations per participant (placed immediately after raw_feature_df is created)
from IPython.display import display

def observations_per_participant(df, id_col='participant_id'):
    """Return participant-level observation counts as a DataFrame with columns [id_col, 'n_obs']"""
    if df is None:
        print('Provided DataFrame is None')
        return pd.DataFrame(columns=[id_col, 'n_obs'])
    if id_col not in df.columns:
        raise ValueError(f"id_col '{id_col}' not found in DataFrame")
    counts = df.groupby(id_col).size().reset_index(name='n_obs')
    # Sort by participant id for deterministic ordering
    counts = counts.sort_values(by=id_col, ascending=True).reset_index(drop=True)
    return counts

# Compute and display counts using the freshly-loaded `raw_feature_df`
obs_counts = observations_per_participant(raw_feature_df)
print(f"Participants: {obs_counts.shape[0]} | Total observations: {len(raw_feature_df)}")
display(obs_counts.tail(50))

Participants: 100 | Total observations: 1088856


,participant_id,n_obs
50,mumblingfoundersubsonic@timestudy_com,8269
51,mysidvattedlexica@timestudy_com,9789
52,neutergoldfishsworn@timestudy_com,8724
53,nicknameoverpaycolt@timestudy_com,6739
54,nucleusbackwatercanning@timestudy_com,13922
55,ogleuncladthermos@timestudy_com,12524
56,orbsquackysyllabuses@timestudy_com,12458
57,pandemicerraticobscure@timestudy_com,12155
58,parakeettrappeddetector@timestudy_com,13005
59,penpalsandbanklifting@timestudy_com,12544


In [75]:
## remove unknown user from df
raw_feature_df = raw_feature_df[raw_feature_df['participant_id'].astype(str).str.lower() != 'unknown_user'].reset_index(drop=True)

In [76]:
# Min and max of days_in_study per participant
from IPython.display import display

def min_max_days_by_participant(df, id_col='participant_id', days_col='days_in_study'):
    """Return a DataFrame with columns [id_col, 'days_min', 'days_max'] for each participant."""
    if df is None:
        print('Provided DataFrame is None')
        return pd.DataFrame(columns=[id_col, 'days_min', 'days_max'])
    if id_col not in df.columns:
        raise ValueError(f"id_col '{id_col}' not found in DataFrame")
    if days_col not in df.columns:
        raise ValueError(f"days_col '{days_col}' not found in DataFrame")
    # Coerce to numeric, preserve NaNs
    days_numeric = pd.to_numeric(df[days_col], errors='coerce')
    tmp = df.copy()
    tmp[days_col] = days_numeric
    agg = tmp.groupby(id_col)[days_col].agg(['min', 'max']).reset_index().rename(columns={'min':'days_min', 'max':'days_max'})
    # Sort by participant id for deterministic output
    agg = agg.sort_values(by=id_col, ascending=True).reset_index(drop=True)
    return agg

# Compute and display the min/max table using the raw feature DataFrame
minmax_days = min_max_days_by_participant(raw_feature_df)
print(f"Participants: {minmax_days.shape[0]} | Total observations: {len(raw_feature_df)}")
display(minmax_days.tail(50))

Participants: 100 | Total observations: 1088856


,participant_id,days_min,days_max
50,mumblingfoundersubsonic@timestudy_com,0,350
51,mysidvattedlexica@timestudy_com,0,333
52,neutergoldfishsworn@timestudy_com,0,273
53,nicknameoverpaycolt@timestudy_com,0,323
54,nucleusbackwatercanning@timestudy_com,0,350
55,ogleuncladthermos@timestudy_com,0,351
56,orbsquackysyllabuses@timestudy_com,0,323
57,pandemicerraticobscure@timestudy_com,0,347
58,parakeettrappeddetector@timestudy_com,0,351
59,penpalsandbanklifting@timestudy_com,0,337


In [77]:
from IPython.display import display

def print_participant_head_tail(df, participant_id, id_col: str = 'participant_id', n: int = 5):
    """
    Simple utility: print the head and tail of `df` for the given participant_id.

    Parameters
    - df: pandas DataFrame containing participant rows
    - participant_id: value of the participant id to filter
    - id_col: name of the participant id column (default 'participant_id')
    - n: number of rows to show from head and tail (default 5)
    """
    if df is None:
        print('Provided DataFrame is None')
        return
    if id_col not in df.columns:
        print(f"id_col '{id_col}' not found in DataFrame columns")
        return
    sub = df[df[id_col] == participant_id]
    if sub.empty:
        print(f"No rows found for {id_col}={participant_id}")
        return
    print(f"--- HEAD ({n}) for participant {participant_id} ---")
    display(sub.head(n))
    print(f"--- TAIL ({n}) for participant {participant_id} ---")
    display(sub.tail(n))

# Example usage (uncomment and replace with a real id):
print_participant_head_tail(raw_feature_df, 'arrivejanitoruniformly@timestudy_com', n=5)


--- HEAD (5) for participant arrivejanitoruniformly@timestudy_com ---


,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
39810,arrivejanitoruniformly@timestudy_com,2021-02-15 12:33:02,0,0,Afternoon,NaN,NaN,Home,0,0.014321,...,746.966667,183.033333,NaN,0,0.00,0.000000,0.000000,0.000000,0.00,0.00
39811,arrivejanitoruniformly@timestudy_com,2021-02-15 12:46:00,0,0,Afternoon,NaN,NaN,Home,0,0.014677,...,734.000000,196.000000,NaN,0,0.00,0.000000,12.966667,0.000000,0.00,0.00
39812,arrivejanitoruniformly@timestudy_com,2021-02-15 12:54:00,0,0,Afternoon,NaN,NaN,Home,0,0.012370,...,726.000000,204.000000,NaN,0,0.00,0.000000,8.000000,0.000000,0.00,0.00
39813,arrivejanitoruniformly@timestudy_com,2021-02-15 13:15:01,1,0,Afternoon,NaN,NaN,Home,0,0.013669,...,704.983333,225.016667,37.567687,0,0.00,0.000000,21.016667,0.000000,0.00,0.00
39814,arrivejanitoruniformly@timestudy_com,2021-02-15 13:33:03,1,0,Afternoon,0.0,0.0,Home,1,0.018931,...,686.950000,243.050000,35.144637,0,0.25,0.333333,18.033333,18.033333,0.25,0.25


--- TAIL (5) for participant arrivejanitoruniformly@timestudy_com ---


,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
50650,arrivejanitoruniformly@timestudy_com,NaN,1,1,NaN,0.0,0.0,Transit,1,3926.818767,...,158.966667,801.033333,41.671243,139,0.931034,1.000000,9.050000,9.050000,0.931034,0.931034
50651,arrivejanitoruniformly@timestudy_com,NaN,1,0,NaN,NaN,NaN,Transit,1,3926.819815,...,122.966667,837.033333,3.311122,140,0.933333,1.000000,28.983333,28.983333,0.933333,0.933333
50652,arrivejanitoruniformly@timestudy_com,NaN,1,0,NaN,NaN,NaN,Transit,1,3926.817126,...,114.983333,845.016667,12.461228,140,0.903226,0.750000,10.000000,38.983333,0.903226,0.903226
50653,arrivejanitoruniformly@timestudy_com,NaN,1,0,NaN,0.0,0.0,Transit,1,3926.818216,...,105.983333,854.016667,75.900521,140,0.875000,0.333333,13.983333,52.966667,0.875000,0.875000
50654,arrivejanitoruniformly@timestudy_com,NaN,1,0,NaN,NaN,NaN,Transit,1,3926.819494,...,95.983333,864.016667,30.008590,140,0.848485,0.000000,8.000000,60.966667,0.848485,0.848485


In [78]:
def missing_value_table(df):
    skip_cols = ['participant_id', 'prompt_time_converted', 'outcome']
    cols = [col for col in df.columns if col.lower() not in skip_cols]
    missing_percent = df[cols].isnull().mean() * 100
    # Also count empty strings as missing
    empty_percent = (df[cols] == '').mean() * 100
    total_missing_percent = missing_percent + empty_percent
    result = pd.DataFrame({
        'missing_%': total_missing_percent.round(2)
    }).sort_values('missing_%', ascending=False)
    return result
    
# Display missing value table
missing_value_table(raw_feature_df)

,missing_%
in_battery_saver_mode,52.14
charging_status,52.14
dist_from_home,15.92
last_phone_usage,12.86
is_phone_locked,12.64
mims_5min,5.03
time_of_day,0.31
closeness_to_sleep_time,0.18
wake_day_part,0.18
closeness_to_wake_time,0.18


In [79]:
def missing_data_by_participant(df):
    skip_cols = ['prompt_time_converted', 'outcome']
    cols = [col for col in df.columns if col.lower() not in skip_cols and col.lower() != 'participant_id']
    # Create a boolean DataFrame for missing values (NaN or empty string)
    missing_bool = df[cols].isnull() | (df[cols] == '')
    # Group by participant_id and calculate % missing for each column
    missing_percent = missing_bool.groupby(df['participant_id']).mean() * 100
    missing_percent = missing_percent.round(2)
    # Reset index to have participant_id as a column
    missing_percent = missing_percent.reset_index()
    return missing_percent

# Print % missing data for each participant and variable
missing_data_by_participant(raw_feature_df)

,participant_id,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,is_phone_locked,last_phone_usage,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
0,afflictedrevenueepilepsy@timestudy_com,0.0,0.00,53.96,53.96,0.0,0.0,19.46,1.41,1.41,...,0.00,0.00,2.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,anagramprobingscrooge@timestudy_com,0.0,0.00,50.58,50.58,0.0,0.0,1.26,0.00,0.00,...,0.00,0.00,3.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,anthillfastinglucrative@timestudy_com,0.0,0.00,52.81,52.81,0.0,0.0,9.21,0.00,0.00,...,0.00,0.00,2.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,arrivejanitoruniformly@timestudy_com,0.0,2.43,52.67,52.67,0.0,0.0,5.28,0.00,0.00,...,0.00,0.00,1.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,attirecrabbinghumbling@timestudy_com,0.0,0.92,52.21,52.21,0.0,0.0,7.11,0.00,0.00,...,0.00,0.00,7.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,vagabondnumerousflatterer@timestudy_com,0.0,0.00,51.35,51.35,0.0,0.0,55.70,22.91,25.51,...,0.54,0.54,7.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,washboardceramicsenticing@timestudy_com,0.0,0.00,52.93,52.93,0.0,0.0,2.25,0.00,0.00,...,0.00,0.00,1.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,wikipediaetchingcrystal@timestudy_com,0.0,0.00,51.45,51.45,0.0,0.0,10.92,0.00,0.00,...,0.00,0.00,1.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,wrigglecatalyststerility@timestudy_com,0.0,0.00,52.13,52.13,0.0,0.0,6.98,0.38,0.38,...,0.00,0.00,2.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## one-hot encoding
Convert the categorical variables into one-hot dummy codes

In [80]:
def one_hot_encode_features(df, columns):
    """
    One-hot encode specified categorical columns in the DataFrame.
    Returns a new DataFrame with one-hot encoded columns as 0/1 integers and original columns dropped.
    """
    df_encoded = df.copy()
    df_encoded = pd.get_dummies(df_encoded, columns=columns, prefix=columns, drop_first=False)
    # Ensure all new one-hot columns are int (0/1)
    for col in df_encoded.columns:
        if any(col.startswith(f'{c}_') for c in columns):
            df_encoded[col] = df_encoded[col].astype(int)
    return df_encoded

# Example usage:
categorical_vars = ['time_of_day', 'location_category', 'wake_day_part']
raw_feature_df_encoded = one_hot_encode_features(raw_feature_df, categorical_vars)
raw_feature_df_encoded.head()

,participant_id,prompt_time_converted,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,...,time_of_day_Night,location_category_Home,location_category_Other,location_category_School,location_category_Transit,location_category_Work,wake_day_part_0.0,wake_day_part_1.0,wake_day_part_2.0,wake_day_part_3.0
0,afflictedrevenueepilepsy@timestudy_com,2021-06-23 06:38:01,0,0,0.0,0.0,0,0.006074,1.0,60.0,...,0,1,0,0,0,0,1,0,0,0
1,afflictedrevenueepilepsy@timestudy_com,2021-06-23 06:46:02,1,0,NaN,NaN,0,0.005902,1.0,60.0,...,0,1,0,0,0,0,1,0,0,0
2,afflictedrevenueepilepsy@timestudy_com,2021-06-23 06:54:04,1,0,0.0,0.0,0,0.005426,1.0,60.0,...,0,1,0,0,0,0,1,0,0,0
3,afflictedrevenueepilepsy@timestudy_com,2021-06-23 07:12:02,0,0,0.0,1.0,0,0.005985,1.0,60.0,...,0,1,0,0,0,0,1,0,0,0
4,afflictedrevenueepilepsy@timestudy_com,2021-06-23 07:23:02,0,0,0.0,1.0,0,0.006400,1.0,60.0,...,0,1,0,0,0,0,1,0,0,0


In [81]:
## List the column names in the DataFrame
def list_column_names(df: pd.DataFrame) -> List[str]:
    """
    List all column names in the DataFrame.
    """
    return df.columns.tolist()

## Use the function to get column names
column_names = list_column_names(raw_feature_df_encoded)
print("Column names in the DataFrame:")
print(column_names)

Column names in the DataFrame:
['participant_id', 'prompt_time_converted', 'outcome', 'is_weekend', 'in_battery_saver_mode', 'charging_status', 'screen_on', 'dist_from_home', 'is_phone_locked', 'last_phone_usage', 'closeness_to_sleep_time', 'closeness_to_wake_time', 'mims_5min', 'days_in_study', 'completion_24h', 'completion_1h', 'time_between_prompts', 'time_since_last_answered', 'completion_since_wake', 'completion_since_start', 'time_of_day_Afternoon', 'time_of_day_Early Morning', 'time_of_day_Evening', 'time_of_day_Late Night', 'time_of_day_Morning', 'time_of_day_Night', 'location_category_Home', 'location_category_Other', 'location_category_School', 'location_category_Transit', 'location_category_Work', 'wake_day_part_0.0', 'wake_day_part_1.0', 'wake_day_part_2.0', 'wake_day_part_3.0']


## Normalize features [Skip this step]

In [27]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scale_by_participant(df, columns, group_col='participant_id'):
    """
    Apply min-max scaling (0-1) to specified columns, grouped by participant_id.
    Returns a new DataFrame with scaled columns (original columns replaced).
    """
    df_scaled = df.copy()
    scaler = MinMaxScaler()
    # Apply scaling for each participant
    for pid, group in df.groupby(group_col):
        idx = group.index
        scaled_values = scaler.fit_transform(group[columns])
        df_scaled.loc[idx, columns] = scaled_values
    return df_scaled

# Example usage:
scale_columns = ['dist_from_home', 'last_phone_usage', 'closeness_to_sleep_time', 'closeness_to_wake_time', 
                 'mims_5min', 'time_between_prompts', 'time_since_last_answered']
raw_feature_df_scaled = min_max_scale_by_participant(raw_feature_df_encoded, scale_columns)
raw_feature_df_scaled[scale_columns + ['participant_id']].head()

/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:686: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:706: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:686: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:706: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_tim

,dist_from_home,last_phone_usage,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,time_between_prompts,time_since_last_answered,participant_id
0,0.000002,0.0,0.187031,0.807168,0.150510,0.000000,0.000000,arrivejanitoruniformly@timestudy_com
1,0.000002,0.0,0.180555,0.814221,0.177391,0.000856,0.000000,arrivejanitoruniformly@timestudy_com
2,0.000002,0.0,0.176050,0.819127,0.220735,0.000528,0.000000,arrivejanitoruniformly@timestudy_com
3,0.000002,0.0,0.172060,0.823473,0.165295,0.001388,0.000000,arrivejanitoruniformly@timestudy_com
4,0.000002,0.0,0.104721,0.896812,0.240907,0.001191,0.001191,arrivejanitoruniformly@timestudy_com


## FixedMax scaling for days in study

In [82]:
# Fixed-max scaling for days_in_study with fixed_max = 365

def fixed_max_scale_days_in_study_365(df: pd.DataFrame,
                                     group_col: str = 'participant_id',
                                     days_col: str = 'days_in_study',
                                     fixed_max: float = 365.0,
                                     inplace: bool = False) -> pd.DataFrame:
    """Scale the `days_col` to [0,1] using a fixed maximum value (default 365).

    This variant overwrites the original `days_col` with the scaled values
    (in-place if requested, otherwise on a copy).

    Parameters
    - df: pandas DataFrame containing the days column
    - group_col: kept for API compatibility (not used)
    - days_col: name of the days-in-study column to scale (will be overwritten)
    - fixed_max: maximum days value to use for scaling
    - inplace: if True, modify `df` in-place and return it; otherwise work on a copy

    Returns
    - pandas DataFrame with `days_col` replaced by scaled values in [0,1]
    """
    if df is None:
        raise ValueError('df must be a pandas DataFrame, got None')
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f'df must be a pandas DataFrame, got {type(df)}')
    if days_col not in df.columns:
        raise ValueError(f"days_col '{days_col}' not found in DataFrame")

    # Work on a copy unless inplace requested
    if not inplace:
        df = df.copy()

    # Coerce to numeric (invalid values -> NaN)
    coerced = pd.to_numeric(df[days_col], errors='coerce')

    # Overwrite the original column with clipped & scaled values; NaNs preserved
    df[days_col] = coerced.clip(lower=0, upper=float(fixed_max)) / float(fixed_max)
    df[days_col] = df[days_col].astype(float)

    return df

raw_feature_df_encoded = fixed_max_scale_days_in_study_365(raw_feature_df_encoded)

In [83]:
## Show the new days_in_study column along with participant_id for preview
raw_feature_df_encoded[['participant_id', 'days_in_study']]

,participant_id,days_in_study
0,afflictedrevenueepilepsy@timestudy_com,0.000000
1,afflictedrevenueepilepsy@timestudy_com,0.000000
2,afflictedrevenueepilepsy@timestudy_com,0.000000
3,afflictedrevenueepilepsy@timestudy_com,0.000000
4,afflictedrevenueepilepsy@timestudy_com,0.000000
...,...,...
1088851,yearlingfiberspotty@timestudy_com,0.934247
1088852,yearlingfiberspotty@timestudy_com,0.934247
1088853,yearlingfiberspotty@timestudy_com,0.934247
1088854,yearlingfiberspotty@timestudy_com,0.934247


## Add missingness indicator

In [84]:
def add_missingness_indicators(df, skip_cols=None, inplace=False):
    """Add binary missingness indicator columns for dataframe columns."""
    if skip_cols is None:
        skip_cols = ['participant_id', 'outcome', 'prompt_time_converted']
    if df is None:
        raise ValueError('df must be a pandas DataFrame, got None')
    if not inplace:
        df = df.copy()
    # Only consider original (non-indicator) columns and skip the requested columns
    cols_to_process = [c for c in df.columns if c not in skip_cols and not str(c).startswith('mi_')]
    for c in cols_to_process:
        new_col = f'mi_{c}'
        # 1 if NaN, 0 otherwise (preserve existing non-NaN values)
        df[new_col] = df[c].isna().astype(int)
    return df

# Example usage: apply to the encoded DataFrame and show the new mi_ columns
if 'raw_feature_df_encoded' in globals():
    raw_feature_df_encoded = add_missingness_indicators(raw_feature_df_encoded)
    mi_cols = [c for c in raw_feature_df_encoded.columns if c.startswith('mi_')][:20]  # preview up to 20 indicators
    print(f'Added {len(mi_cols)} missingness indicator columns (previewing up to 20):')
    display(raw_feature_df_encoded[['participant_id'] + mi_cols].head())
else:
    print('raw_feature_df_encoded not found; run earlier cells to create it first.')

Added 20 missingness indicator columns (previewing up to 20):


,participant_id,mi_is_weekend,mi_in_battery_saver_mode,mi_charging_status,mi_screen_on,mi_dist_from_home,mi_is_phone_locked,mi_last_phone_usage,mi_closeness_to_sleep_time,mi_closeness_to_wake_time,...,mi_days_in_study,mi_completion_24h,mi_completion_1h,mi_time_between_prompts,mi_time_since_last_answered,mi_completion_since_wake,mi_completion_since_start,mi_time_of_day_Afternoon,mi_time_of_day_Early Morning,mi_time_of_day_Evening
0,afflictedrevenueepilepsy@timestudy_com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,afflictedrevenueepilepsy@timestudy_com,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,afflictedrevenueepilepsy@timestudy_com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,afflictedrevenueepilepsy@timestudy_com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,afflictedrevenueepilepsy@timestudy_com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
raw_feature_df_encoded[['participant_id', 'in_battery_saver_mode', 'mi_in_battery_saver_mode']].head()

,participant_id,in_battery_saver_mode,mi_in_battery_saver_mode
0,afflictedrevenueepilepsy@timestudy_com,0.0,0
1,afflictedrevenueepilepsy@timestudy_com,NaN,1
2,afflictedrevenueepilepsy@timestudy_com,0.0,0
3,afflictedrevenueepilepsy@timestudy_com,0.0,0
4,afflictedrevenueepilepsy@timestudy_com,0.0,0


In [86]:
raw_feature_df_encoded[['participant_id', 'wake_day_part_0.0', 'mi_wake_day_part_0.0']].head()

,participant_id,wake_day_part_0.0,mi_wake_day_part_0.0
0,afflictedrevenueepilepsy@timestudy_com,1,0
1,afflictedrevenueepilepsy@timestudy_com,1,0
2,afflictedrevenueepilepsy@timestudy_com,1,0
3,afflictedrevenueepilepsy@timestudy_com,1,0
4,afflictedrevenueepilepsy@timestudy_com,1,0


In [87]:
raw_feature_df_encoded.columns

Index(['participant_id', 'prompt_time_converted', 'outcome', 'is_weekend',
       'in_battery_saver_mode', 'charging_status', 'screen_on',
       'dist_from_home', 'is_phone_locked', 'last_phone_usage',
       'closeness_to_sleep_time', 'closeness_to_wake_time', 'mims_5min',
       'days_in_study', 'completion_24h', 'completion_1h',
       'time_between_prompts', 'time_since_last_answered',
       'completion_since_wake', 'completion_since_start',
       'time_of_day_Afternoon', 'time_of_day_Early Morning',
       'time_of_day_Evening', 'time_of_day_Late Night', 'time_of_day_Morning',
       'time_of_day_Night', 'location_category_Home',
       'location_category_Other', 'location_category_School',
       'location_category_Transit', 'location_category_Work',
       'wake_day_part_0.0', 'wake_day_part_1.0', 'wake_day_part_2.0',
       'wake_day_part_3.0', 'mi_is_weekend', 'mi_in_battery_saver_mode',
       'mi_charging_status', 'mi_screen_on', 'mi_dist_from_home',
       'mi_is_phone_l

In [88]:
# Remove `prompt_time_converted` from `raw_feature_df_encoded` if present
if 'raw_feature_df_encoded' in globals():
    if 'prompt_time_converted' in raw_feature_df_encoded.columns:
        raw_feature_df_encoded = raw_feature_df_encoded.drop(columns=['prompt_time_converted'])
        print("Dropped column 'prompt_time_converted' from raw_feature_df_encoded.")
    else:
        print("Column 'prompt_time_converted' not present in raw_feature_df_encoded; nothing to do.")
else:
    print("raw_feature_df_encoded not found in the notebook namespace. Run the one-hot encoding cell first.")

Dropped column 'prompt_time_converted' from raw_feature_df_encoded.


In [ ]:
## Save the raw_features_df_encoded to a file
## for held out dataset, save to: /Users/adityaponnada/Downloads/time_study_data/processed_features_heldout.csv
## for traning data, save to: /Users/adityaponnada/Downloads/time_study_data/processed_features_rnn.csv
raw_feature_df_encoded.to_csv('/Users/adityaponnada/Downloads/time_study_data/processed_features_rnn.csv', index=False)